# SESION 4 DICIEMBRE

# Modificamos columnas que eran object a float 

In [ ]:
import pandas as pd
import numpy as np
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df_rawdata = pd.read_csv("../Data/raw_data_20231201.csv", index_col=0)

In [ ]:
def cambiar_comas(cadena):
    try:
        return float(cadena.replace(",", "."))
    except:
        return np.nan
    
df_rawdata["Monthlyincome"] = df_rawdata["Monthlyincome"].apply(cambiar_comas)
df_rawdata["Performancerating"] = df_rawdata["Performancerating"].apply(cambiar_comas)
df_rawdata["Totalworkingyears"] = df_rawdata["Totalworkingyears"].apply(cambiar_comas)

In [ ]:
df_rawdata["Employeenumber"]= df_rawdata["Employeenumber"].apply(cambiar_comas)

In [ ]:
df_rawdata["Employeenumber"].isnull().sum()

# Gestión de nulos

Cambiamos los nulos de Employeenumber con un "auto-incremental"

In [ ]:
# Encuentra el último ID existente
ultimo_id = df_rawdata['Employeenumber'].max()
# Define una variable para el siguiente ID después del último
siguiente_id = ultimo_id + 1
# Itera sobre los índices del DataFrame
for indice in df_rawdata.index:
    if pd.isnull(df_rawdata.at[indice, 'Employeenumber']):  # Verifica si el valor es nulo
        df_rawdata.at[indice, 'Employeenumber'] = siguiente_id  # Asigna el siguiente ID
        siguiente_id += 1  # Incrementa el siguiente ID
df_rawdata["Employeenumber"] = df_rawdata["Employeenumber"].astype(int)
# Muestra el DataFrame actualizado
print(df_rawdata["Employeenumber"].tail(10))

In [ ]:
# Muestra el DataFrame actualizado
df_rawdata["Employeenumber"].dtype

In [ ]:
nulls_cat = df_rawdata[df_rawdata.columns[df_rawdata.isnull().any()]].select_dtypes(include = "O").columns
nulls_cat

In [ ]:
(df_rawdata[['Businesstravel', 'Department', 'Educationfield', 'Maritalstatus','Overtime']].isnull().sum() / df_rawdata.shape[0]) * 100

In [ ]:
# sacamos el 'value_counts()' de cada una de las columnas categóricas que tienen nulos para saber como es la distribución de sus categorías
for col in nulls_cat:
    print(f"La distribución de las categorías para la columna {col.upper()}")
    display(df_rawdata[col].value_counts() / df_rawdata.shape[0])
    print("........................")

In [ ]:
nulls_cat = ['Businesstravel', 'Educationfield', 'Maritalstatus', 'Overtime']

Todos desconocidos!

In [ ]:
for columna in nulls_cat:
    # reemplazamos los nulos por el valor Unknown para cada una de las columnas de la lista
    df_rawdata[columna] = df_rawdata[columna].fillna("Unknown")

In [ ]:
df_rawdata["Overtime"].unique()

# Columna Department: vinculada con Jobrole

In [ ]:
# primero vemos cómo está la columna Jobrole
#df_rawdata["Jobrole"].head()

In [ ]:
# # capitalize valores de Jobrole (primera letra de cada palabra)
df_rawdata['Jobrole'] = df_rawdata['Jobrole'].apply(lambda x: ' '.join(word.capitalize() for word in x.lower().split()))
df_rawdata["Jobrole"].head()

In [ ]:
# hacemos función para asignar los valores nulos en Department según la columna Jobrole
def fill_department(df, jobrole, department):
    df.loc[df['Jobrole'] == jobrole, 'Department'] = df.loc[df['Jobrole'] == jobrole, 'Department'].fillna(department)
    return df 

# Uso de la función fill_department para reemplazar los valores faltantes en 'Department'
df_rawdata = fill_department(df_rawdata, 'Research Director', 'Research & Development')
df_rawdata = fill_department(df_rawdata, 'Healthcare Representative', 'Research & Development')
df_rawdata = fill_department(df_rawdata, 'Laboratory Technician', 'Research & Development')
df_rawdata = fill_department(df_rawdata, 'Manufacturing Director', 'Research & Development')
df_rawdata = fill_department(df_rawdata, 'Research Scientist', 'Research & Development')
df_rawdata = fill_department(df_rawdata, 'Sales Executive', 'Sales')
df_rawdata = fill_department(df_rawdata, 'Sales Representative', 'Sales')
df_rawdata = fill_department(df_rawdata, 'Human Resources', 'Human Resources')
df_rawdata = fill_department(df_rawdata, 'Manager', 'Unknown')

# Mostrar el DataFrame actualizado
df_rawdata[["Jobrole", "Department"]].head(37)

In [ ]:
# Mostrar estadísticas descriptivas y conteo de valores únicos
print("Estadísticas descriptivas para 'Jobrole':\n", df_rawdata['Jobrole'].describe())
print("\nEstadísticas descriptivas para 'Department':\n", df_rawdata['Department'].describe())

print("\nConteo de valores únicos para 'Jobrole':\n", df_rawdata['Jobrole'].value_counts())
print("\nConteo de valores únicos para 'Department':\n", df_rawdata['Department'].value_counts())

# Todas las columnas categóricas listas 

In [ ]:
# Como hay valores negativos en la columna Distancefromhome los cambiamos a nulos.
def negative_to_null(data):
    if data < 0:
        return np.nan
    else:
        return data
    
df_rawdata["Distancefromhome"] = df_rawdata["Distancefromhome"].apply(negative_to_null)

In [ ]:
df_rawdata["Distancefromhome"].tail(5)

## Gestión nulos columnas numéricas

In [ ]:
nulls_num = df_rawdata[df_rawdata.columns[df_rawdata.isna().any()]].columns
nulls_num

In [ ]:
# Función para quitar "Not Available" y poner nulos en su lugar
def not_available_to_null(data):
    if data == 'Not Available':
        return np.nan
    else:
        return float(data)
df_rawdata["Hourlyrate"] = df_rawdata["Hourlyrate"].apply(not_available_to_null)

In [ ]:
df_rawdata_copy = df_rawdata.copy()

In [ ]:
print(f"Tenemos nulos: \n{df_rawdata_copy[['Dailyrate', 'Distancefromhome', 'Monthlyincome', 'Performancerating','Totalworkingyears', 'Worklifebalance', 'Hourlyrate']].isnull().sum()} nulos")			

# IterativeImputer

In [ ]:
# instanciamos las clases										
imputer_iterative = IterativeImputer(max_iter = 20, random_state = 42)			
# ajustamos y tranformamos los datos										
imputer_iterative_imputado = imputer_iterative.fit_transform(df_rawdata_copy[['Dailyrate', 'Distancefromhome', 'Monthlyincome', 'Performancerating','Totalworkingyears', 'Worklifebalance', 'Hourlyrate']])
# comprobamos que es lo que nos devuelve, que en este caso es un array también					
imputer_iterative_imputado																						

In [ ]:
df_rawdata_copy[['Dailyrate_ITE', 'Distancefromhome_ITE', 'Monthlyincome_ITE', 'Performancerating_ITE','Totalworkingyears_ITE', 'Worklifebalance_ITE', 'Hourlyrate_ITE']] = imputer_iterative_imputado										
										
# comprobamos los nulos										
print(f"Después del 'Iterative' tenemos: \n{df_rawdata_copy[['Dailyrate_ITE', 'Distancefromhome_ITE', 'Monthlyincome_ITE', 'Performancerating_ITE', 'Totalworkingyears_ITE', 'Worklifebalance_ITE', 'Hourlyrate_ITE']].isnull().sum()} nulos")										
										

# KNN Imputer

In [ ]:
# instanciamos la clase del KNNImputer
imputer_knn = KNNImputer(n_neighbors = 5)
# ajustamos y transformamos los datos
imputer_knn_imputado = imputer_knn.fit_transform(df_rawdata_copy[['Dailyrate', 'Distancefromhome', 'Monthlyincome', 'Performancerating','Totalworkingyears', 'Worklifebalance', 'Hourlyrate']])
# comprobamos que es lo que nos devuelve, que sigue siendo un array
imputer_knn_imputado

In [ ]:
# por último nos queda añadir ese array al DataFrame como hemos hecho hasta ahora
df_rawdata_copy[['Dailyrate_KNN', 'Distancefromhome_KNN', 'Monthlyincome_KNN', 'Performancerating_KNN','Totalworkingyears_KNN', 'Worklifebalance_KNN', 'Hourlyrate_KNN']] = imputer_knn_imputado

# comprobamos los nulos
print(f"Después del 'KNN' tenemos: \n{df_rawdata_copy[['Dailyrate_KNN', 'Distancefromhome_KNN', 'Monthlyincome_KNN', 'Performancerating_KNN','Totalworkingyears_KNN', 'Worklifebalance_KNN', 'Hourlyrate_KNN']].isnull().sum()} nulos")


In [ ]:
df_rawdata_copy.describe()[['Dailyrate', 'Dailyrate_ITE', 'Dailyrate_KNN']]

In [ ]:
df_rawdata_copy.describe()[["Distancefromhome", "Distancefromhome_ITE", "Distancefromhome_KNN"]]

In [ ]:
df_rawdata_copy.describe()[["Monthlyincome", "Monthlyincome_ITE", "Monthlyincome_KNN"]]

In [ ]:
df_rawdata_copy.describe()[["Performancerating", "Performancerating_ITE", "Performancerating_KNN"]]

In [ ]:
df_rawdata_copy.describe()[["Totalworkingyears","Totalworkingyears_ITE", "Totalworkingyears_KNN"]]

In [ ]:
df_rawdata_copy.describe()[["Worklifebalance", "Worklifebalance_ITE", "Worklifebalance_KNN"]]

In [ ]:
df_rawdata_copy.describe()[["Hourlyrate","Hourlyrate_ITE","Hourlyrate_KNN"]]

Dailyrate_KNN, Distancefromhome_KNN, Monthlyincome_KNN, Performancerating_KNN, Totalworkingyears_KNN, Worklifebalance_KNN, Hourlyrate_KNN

# Hemos seleccionado el método KNN Imputer ya que los resultados son más precisos.


In [ ]:
#al final me quedo con las del método kn por escoger alguna
df_rawdata_copy.drop(['Dailyrate', 'Dailyrate_ITE', 'Distancefromhome', 'Distancefromhome_ITE', 'Monthlyincome', 'Monthlyincome_ITE', 'Performancerating', 'Performancerating_ITE', 'Totalworkingyears', 'Totalworkingyears_ITE', 'Worklifebalance', 'Worklifebalance_ITE', 'Hourlyrate', 'Hourlyrate_ITE'], axis = 1, inplace = True)

# ahora vamos a cambiar el nombre de las columnas que quedaron para que tengan el mismo nombre de origen
nuevo_nombre = {"Dailyrate_KNN": "Dailyrate",  'Distancefromhome_KNN': "Distancefromhome", "Monthlyincome_KNN": "Monthlyincome", "Performancerating_KNN": "Performancerating","Totalworkingyears_KNN": "Totalworkingyears", "Worklifebalance_KNN": "Worklifebalance", "Hourlyrate_KNN":"Hourlyrate"}
df_rawdata_copy.rename(columns = nuevo_nombre, inplace = True)


In [ ]:
df_rawdata_copy.columns

In [ ]:
df_rawdata_copy.to_csv("../Data/raw_data_final_20231205.csv")